# Integrated model for routing pollsters and vehicles

<div class="alert alert-block alert-info">
This notebook contains the iterative code to build and solve the Integrated Vehicles and Polsters Routing Problem (IVPRP) over the test set designed and presented for publication.
</div>

In [1]:
from gurobipy import *

In [2]:
# Packages
import numpy  as np
import pandas as pd
import time
from collections import deque

In [3]:
# Aliases
append, arange, around, asarray, loadtxt, zeros = np.append, np.arange, np.around, np.asarray, np.loadtxt, np.zeros
DataFrame, concat = pd.DataFrame, pd.concat

In [4]:
# General functions for reading and writing
def Reader(nom):    return loadtxt('Instances/'+ nom + '.txt' )
def P_Reader(n):    return  Reader('Pollster_' + str(n))
def V_Reader(n):    return  Reader( 'Vehicle_' + str(n))
def S_Reader(n):    return  Reader( 'Service_' + str(n))
def T_Reader(n):    return  Reader(    'Time_' + str(n))

def Write(df,nom):  return DataFrame(df).to_csv('Instances/'+ nom +'.txt', header=None, index=None, sep=' ', mode='a')

## Basics

<div class="alert alert-block alert-warning">
A general list is needed to call and run all the IP's at once. This list will store:
    
* $n$ is the number of stores to be visited,
* $E$ is the set of available pollsters,
* $K$ is the set of available vehicles,
* $Q$ is the vehicle's capacity,
* $S$ is the number of days.
</div>

In [5]:
#      (n, K, E, S, Q)
Γ = ( (10, 3, 3, 1, 1), 
      (12, 2, 2, 2, 1),
      (14, 2, 2, 2, 1),
      (16, 2, 3, 2, 2),
      (18, 2, 3, 2, 2),
      (20, 3, 4, 1, 2),
      (25, 2, 2, 2, 3),
      (30, 2, 2, 2, 3),
      (40, 2, 4, 2, 4),
      (50, 3, 5, 2, 4)
    )

# Iterative code

<div class="alert alert-block alert-warning">
At last costs are fixed in advance, as they are not instance dependent:
    
* $\kappa_0$ is the daily cost of operations,
* $\kappa_1$ is the cost of hiring a vehicle,
* $\kappa_2$ is the cost of hiring a pollster.
</div>

In [6]:
κ_0 = 200.0
κ_1 = 100.0
κ_2 = 40.0

<div class="alert alert-block alert-success">
Using the generator created above, we build the basic index sets to build the model and read data.
</div>
<div class="alert alert-block alert-warning">
The read matrices are processed:
    
* $d$ captures pollstering times,
* $t$ is the time that a pollster takes to walk among pairs of stores,
* $\tau$ is the time that vehicles take to move between pairs of stores,

* $[\rho_0,\rho_1]$ is the time window for breaks,
* $P$ is the length of the pause,
* $\beta$ is the time horizon limit.
</div>

<div class="alert alert-block alert-warning">
The multigraph is built and then the model is initiated.
</div>

In [9]:
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    print('*** Solving instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    
    '''Multigraph'''
    m = 2*n

    # Domain for vehicles.                         Graph size:    (2*n) * (n + 1) + (2*n-1) * n = 4*(n**2) + n
    dom_v  = [ (0,j)   for j in range(1,m+1) ] 
    dom_v += [ (j,m+1) for j in range(1,m+1) ]
    dom_v += [ (i,j)   for i in range(1,m+1) for j in range(1,m+1) if j not in [i,i-n]]

    # Domain for pollsters.                        Graph size:    (n-1)**2 + (n-1) + n = n**2
    dom_e  = [ (i,i+n) for i in range(  1,n+1) ]
    dom_e += [ (i,j)   for i in range(1+n,m+1) for j in range(1,n+1) if j!=i-n ]
    
    C_minus, C_plus, C_0, C_m, C = range(1,n+1), range(n+1,2*n+1),  range(0,n+1), range(1,2*n+2),  range(1,2*n+1)
    
    '''MIP model'''
    mo = Model()
    x, y, z, b, f, w, B, u = {}, {}, {}, {}, {}, {}, {}, {}
    
    # Pollster variables
    ## Nodes
    b = mo.addVars( C_minus, E, S, vtype = 'B', name ='b')             # **begining**    n * |S|*|E|
    f = mo.addVars( C_plus,  E, S, vtype = 'B', name ='f')             # **ending**      n * |S|*|E|
    w = mo.addVars( C_minus, E, S, vtype = 'B', name ='w')             # **break**       n * |S|*|E|
    ## Arcs
    x = mo.addVars( dom_e, E, S, vtype = 'B', name = 'x')              # **Walking paths**     n^2 * |S|*|E|
    # Vehicle variables
    y = mo.addVars( dom_v, K, S, vtype = 'B', name = 'y')              # **Vehicle-paths**     (4*n^2+n)*|K|*|S|
    z = mo.addVars( dom_v, E, S, vtype = 'B', name = 'z')              # **t-paths**           (4*n^2+n)*|E|*|S|
    # Time and days variables
    B = mo.addVars( C, vtype = 'C', name = 'B', ub  = β )              # **In-Out timing**     2*n
    u = mo.addVars( S, vtype = 'B', name = 'u', obj = κ_0)             # **Day**               |S|

    mo.update()
    
    deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
    deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
    mo.setAttr('ModelSense', GRB.MINIMIZE)
    mo.update()
    
    
    start = time.time()
    
    # x & z vars interaction
    # 1a - Exclusive attention:      sum x[i,i+n] = 1
    mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');
    # 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
    mo.addConstrs( (x.sum('*',i,e,s) == x[i,i+n,e,s] - b[i,e,s] for i in C_minus for s in S for e in E), name='R-1b')
    mo.addConstrs( (x.sum(i,'*',e,s) == x[i-n,i,e,s] - f[i,e,s] for i in C_plus for s in S for e in E), name='R-1c')
    # 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
    mo.addConstrs( (z.sum('*',i,e,s) <= 1.0 - x[i,i+n,e,s] + b[i,e,s] for i in C_minus for s in S for e in E), name='R-1d')
    mo.addConstrs( (z.sum(i,'*',e,s) <= 1.0 - x[i-n,i,e,s] + f[i,e,s] for i in C_plus for s in S for e in E), name='R-1e')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == b[i,e,s] for i in C_minus for s in S for e in E), name='R-1f')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == -f[i,e,s] for i in C_plus for s in S for e in E), name='R-1g')
    # 1h – One trip per day for each pollster
    mo.addConstrs( (z.sum(0,'*',e,s) <= u[s] for s in S for e in E), name='R-1h')
    

    # y & z vars interaction
    start = time.time()
    # 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
    mo.addConstrs( (y.sum(i,'*','*',s) == b.sum(i,'*',s) for i in C_minus for s in S), name='R-2a')
    mo.addConstrs( (y.sum(i,'*','*',s) == f.sum(i,'*',s) for i in C_plus for s in S), name='R-2b')
    # 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
    mo.addConstrs( (y.sum('*',i,k,s) == y.sum(i,'*',k,s) for i in C for k in K for s in S ), name='R-2c')
    mo.addConstrs( (y.sum('*',2*n+1,k,s) == y.sum(0,'*',k,s) for k in K for s in S ), name='R-2d')
    # 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
    mo.addConstrs( (y.sum(0,'*',k,s) <= u[s] for k in K for s in S ), name='R-2e')
    # 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
    mo.addConstrs( (z.sum(i,j,'*',s) <= Q*y.sum(i,j,'*',s) for (i,j) in dom_v for s in S ), name='R-2f')

    
    # B vars enforce connected paths
    M = 1e+4
    # 3a,b – Arriving marker:  B[j] >= B[i] + t[i,j] + sum_s w[i] P - M(1 - sum_s x[i,j])
    mo.addConstrs( (B[i+n] - B[i] - d[i] >= P*w.sum(i,'*') for i in C_minus ), name='R-3a')
    mo.addConstrs( (B[j] - B[i+n] - t[i-1,j-1] >= -M*(1.0 - x.sum(i+n,j,'*')) 
                   for i in C_minus for j in C_minus if j!=i ), name='R-3b')
    ## Trivial
    mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
    # 3c — Arrival after transport: B[j] >= B[i] + τ_{i,j} - M(1 - sum_{s,k} y[i,j] )
    mo.addConstrs( 
        (B[j] - B[i] + M*(1.0 - y.sum(i,j,'*')) >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)] 
         for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c')
    # 3d — First transportation: B[i] >= tau_{0,i} - M(1 - sum_{s,k} y[0,i]^{k,s} )
    mo.addConstrs( (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)] - β*(1.0 - y.sum(0,i,'*','*')) 
                   for i in C ), name='R-3d')

    # 3e — Arrival marks: B[2n+1]^{s} >= B[i] + tau_{i,2n+1} - M(1-y[i,2n+1])
    mo.addConstrs( ( β * u[s] - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] - M*(1.0 - y.sum(i,2*n+1,'*',s))
                   for i in C for s in S ), name='R-3e')
    
    # w vars interact
    # 4a — Breaks TW:   p0 sum_{e,s} w[i] <= B[i] + d[i] <= p1 + M( 1 - sum_s w[i] )
    mo.addConstrs( (ρ_0*w.sum(i,'*') - B[i] - d[i] <= 0.0 for i in C_minus ), name='R-4a0')
    mo.addConstrs( (B[i] + d[i] - ρ_1 - β*(1.0 - w.sum(i,'*')) <= 0.0 for i in C_minus ), name='R-4a1')
    # 4b – One break per pollster:     w[i] <=  x[i,j]
    mo.addConstrs( (w[i,e,s] <= x[i,i+n,e,s] for i in C_minus for e in E for s in S ), name='R-4b')
    # 5c — Mandatory breaks:   sum_i w[i] = sum_j z[0,j]
    mo.addConstrs( (w.sum('*',e,s) == z.sum(0,'*',e,s) for e in E for s in S ), name='R-4b')

    
    # More

    mo.addConstr( z.sum(0,'*',0,0) == 1 , name='R-5a')
    mo.addConstr( y.sum(0,'*',0,0) == 1 , name='R-5d')

    if E.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e-1,s) for e in E for s in S if e > 0 ), name='R-5b')

        if S.size > 1:
            mo.addConstrs( 
                (b.sum('*',e,s) <= n - quicksum(b[i,e-1,r] for i in C_minus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5k')
            mo.addConstrs( 
                (f.sum('*',e,s) <= n - quicksum(f[i,e-1,r] for i in C_plus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5l')

    if S.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e,s-1) for e in E for s in S if s > 0 ), name='R-5c')
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k,s-1) for k in K for s in S if s > 0 ), name='R-5f')
        mo.addConstrs( (b.sum(i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5g')
        mo.addConstrs( (f.sum(i,'*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5h')
        mo.addConstrs( (x.sum('*',i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5i')
        mo.addConstrs( (x.sum(i,'*','*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5j')
        mo.addConstrs( (u[s] <= u[s-1] for s in S if s > 0), name='R-5m')

    if K.size > 1:
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k-1,s) for k in K for s in S if k > 0 ), name='R-5e')

    end = time.time()
    print('Constraints took {} seconds.'.format(end-start))
    
    Λ   = [ np.append(a, arange(a.size + b + 1, (b+1)*a.size + 1) ) for b,a in enumerate([(E+1)+s for s in S]) ]
    Λ_β = {λ: d.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d.sum()/λ <= β - P}

    F_2, T_1 = max(Λ_β.items(), key=lambda x: x[1]) 
    F_1 = next(a for a,b in enumerate(Λ) if F_2 in b) + 1
    F_3 = [k for k in arange(F_1, K.size * F_1 + 1) if F_2 <= Q * k].pop(0)
    print('Lower bounds found: |S| >=', F_1, '|K| >=',F_3, '|E| >=', F_2)

    # Additional bound over the number of days
    mo.addConstr( u.sum() >= F_1)
    # Additional bound over the number of required pollsters
    mo.addConstr(z.sum(0,'*','*','*') >= F_2)
    # Additional bound over the number of required vehicles
    mo.addConstr(y.sum(0,'*','*','*') >= F_3)

    if S.size > 1:
        mo.addConstrs( (z.sum(0,'*',0,s) == u[s] for s in S) )
        mo.addConstrs( (y.sum(0,'*',0,s) == u[s] for s in S) );

    mo.addConstrs( (y.sum(0,'*','*',s) <= y.sum(0,'*','*',s-1) for s in S if s>0) );
    deque( (v.setAttr('BranchPriority', 10) for v in y.values()), 0);
    
    print('Model ready')
    mo.update()
    
    '''Optimization'''
    # Parameters
    '''
    mo.Params.MIPFocus = 1;    
    mo.Params.Heuristics = 0.33
    mo.Params.Cuts = 3;      #<- w/o finds some feasible solutions 
    mo.Params.Method = 2

    mo.Params.SimplexPricing = 3
    mo.Params.CutAggPasses = 12;    mo.Params.CutPasses = 12;   mo.Params.PrePasses = 8

    mo.Params.ImproveStartTime = 100


    mo.setParam('Presolve', 2)
    mo.Params.GURO_PAR_PREPROBE = 3
    mo.Params.PreSparsify = 1
    mo.Params.PrePasses = 500 # best solution so far w/ 500
    '''
    mo.Params.VarBranch = 1
    mo.Params.BranchDir = 0
    
    # First round
    mo.Params.TimeLimit = 1000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')
        
        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})
        
        
        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')
        
        Out_File = 'Instances/Results_'+ str(n) +'-1000.xlsx'
        
        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)
        
        print('Solution stored in local folder.')
    
    print('\n')
    # Second round
    mo.Params.ImproveStartTime = 1000
    mo.Params.TimeLimit = 2000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')

        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})


        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')

        Out_File = 'Instances/Results_'+ str(n) +'-2000.xlsx'

        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)

        print('Solution stored in local folder.')
    else:
        print('No feasible solution found after 3000s.')
    
    
    disposeDefaultEnv()
    mo.dispose()
    del mo, x, y, z, b, f, w, B, u
    print('\n\n')

*** Solving instance of size: 10  ***

Using license file /Users/andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 0.08693981170654297 seconds.
Lower bounds found: |S| >= 1 |K| >= 2 |E| >= 2
Model ready
Changed value of parameter VarBranch to 1
   Prev: -1  Min: -1  Max: 3  Default: -1
Parameter BranchDir unchanged
   Value: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1271 rows, 2871 columns and 13727 nonzeros
Model fingerprint: 0xe739f6a0
Variable types: 20 continuous, 2851 integer (2851 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+04]
Presolve removed 15 rows and 1 columns
Presolve time: 0.09s
Presolved: 1256 rows, 2870 columns, 13480 nonzeros
Variable types: 20 continuous, 2850 int

 10111  4742  480.00000  119   60  920.00000  480.00000  47.8%  66.8   91s
 10376  4914  480.00000  212   70  920.00000  480.00000  47.8%  67.2   95s
 11109  5590  480.00000   92   55  920.00000  480.00000  47.8%  67.9  100s
 12095  6179  480.00000  112   69  920.00000  480.00000  47.8%  70.3  105s
 13385  6820  480.00000   93   17  920.00000  480.00000  47.8%  72.0  110s
 14138  7479  480.00000  131   23  920.00000  480.00000  47.8%  73.0  115s
 16296  7707  480.00000  122  102  920.00000  480.00000  47.8%  69.5  120s
H16535  7661                     820.0000000  480.00000  41.5%  69.3  123s
 17268  7851  480.00000   50   52  820.00000  480.00000  41.5%  68.8  125s
 18802  8817  480.00000   56   53  820.00000  480.00000  41.5%  71.4  131s
 19607  9588  480.00000  183   74  820.00000  480.00000  41.5%  72.4  136s
 21490 10463  480.00000  148   54  820.00000  480.00000  41.5%  71.6  149s
 21778 10908  480.00000  172   32  820.00000  480.00000  41.5%  72.0  151s
 23578 11390 infeasible  

 169731 56151  480.00000  172   86  820.00000  480.00000  41.5%  52.4  635s
 172187 56675 infeasible  172       820.00000  480.00000  41.5%  52.1  641s
 173025 56852  480.00000  153  102  820.00000  480.00000  41.5%  52.1  646s
 174162 57196  680.00000  237   45  820.00000  480.00000  41.5%  52.1  650s
 175272 57563 infeasible  206       820.00000  480.00000  41.5%  52.1  655s
 177865 58011 infeasible  217       820.00000  480.00000  41.5%  51.9  660s
 179020 58348 infeasible  214       820.00000  480.00000  41.5%  51.9  665s
 180305 58550  480.00000  211   19  820.00000  480.00000  41.5%  51.9  671s
 181187 58687  480.00000  153   46  820.00000  480.00000  41.5%  51.9  675s
 181975 58999  480.20507  198   79  820.00000  480.00000  41.5%  52.0  680s
 183467 59465  780.00000  219   28  820.00000  480.00000  41.5%  52.0  686s
 185655 59467 infeasible  195       820.00000  480.00000  41.5%  51.8  691s
 186021 59542  480.00000  202   22  820.00000  480.00000  41.5%  51.8  695s
 187707 5995

 259876 74404 infeasible  171       820.00000  480.00000  41.5%  57.5 1095s
 260612 74407  480.00000  176   30  820.00000  480.00000  41.5%  57.5 1100s
 262031 74519  480.00000  141   29  820.00000  480.00000  41.5%  57.5 1105s
 262854 74720  480.00000  162   94  820.00000  480.00000  41.5%  57.6 1111s
 263757 74730  480.36220  154  102  820.00000  480.00000  41.5%  57.6 1117s
 264173 74768 infeasible  157       820.00000  480.00000  41.5%  57.6 1120s
 265214 74898  480.00000  152   60  820.00000  480.00000  41.5%  57.7 1125s
 266168 74935 infeasible  151       820.00000  480.00000  41.5%  57.7 1131s
 267059 75152  480.00000  170   57  820.00000  480.00000  41.5%  57.8 1137s
 267860 75139 infeasible  182       820.00000  480.00000  41.5%  57.7 1140s
 268618 75205  480.00000  152   89  820.00000  480.00000  41.5%  57.8 1146s
 270592 75436  680.26395  170   68  820.00000  480.00000  41.5%  57.7 1152s
 271164 75545 infeasible  158       820.00000  480.00000  41.5%  57.8 1157s
 271566 7560

   Value: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3327 rows, 7366 columns and 35800 nonzeros
Model fingerprint: 0xfccdf279
Variable types: 28 continuous, 7338 integer (7338 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+04]
Presolve added 0 rows and 3 columns
Presolve removed 33 rows and 0 columns
Presolve time: 0.09s
Presolved: 3294 rows, 7369 columns, 34514 nonzeros
Variable types: 28 continuous, 7341 integer (7341 binary)

Root relaxation: objective 4.800000e+02, 4945 iterations, 0.76 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  480.00000    0   27          -  480.00000      -     -

...

 23408 10873  380.00000   56  239          -  380.00000      -   444 1603s
 23681 11065 infeasible   81               -  380.00000      -   445 1614s
 24187 11182  380.00000   54  204          -  380.00000      -   441 1627s
 24451 11280 infeasible   47               -  380.00000      -   441 1639s
 24729 11327 infeasible   56               -  380.00000      -   441 1650s
 24814 11388  380.00000   61  182          -  380.00000      -   444 1662s
 24985 11455  380.00000   60   61          -  380.00000      -   444 1676s
 25115 11521 infeasible   84               -  380.00000      -   447 1687s
 25284 11542  380.00000   79  138          -  380.00000      -   448 1699s
 25367 11580  380.00000   32  297          -  380.00000      -   449 1711s
 25409 11650  380.00000   41  185          -  380.00000      -   453 1723s
 25513 11731  380.00000   50   28          -  380.00000      -   456 1735s
 25646 11829  525.00000  101  145          -  380.00000      -   459 1748s
 25887 11987 infeasible  

 57719 26654  860.00000   79  217          -  380.00000      -   399 2778s
 58480 26875  680.00000   82  135          -  380.00000      -   396 2784s
 58994 27153  960.00000   85   81          -  380.00000      -   393 2791s
 59583 27312 infeasible   93               -  380.00000      -   391 2804s
 59961 27519  860.00000   86  224          -  380.00000      -   390 2810s
 60388 27638 infeasible   95               -  380.00000      -   389 2816s
 60671 27810  580.00000   72  177          -  380.00000      -   388 2822s
 60958 27882  760.00000   66   54          -  380.00000      -   388 2828s
 61128 27909  380.00000   58  175          -  380.00000      -   388 2834s
 61178 27992  380.00000   66  235          -  380.00000      -   388 2840s
 61357 28108  380.00000   42  236          -  380.00000      -   388 2846s
 61534 28198  380.00000   50   41          -  380.00000      -   389 2853s
 61710 28261  380.00000   54  223          -  380.00000      -   389 2859s
 61864 28336 infeasible  

   937   604  960.00000  123  248          -  380.00000      -  8118 1296s
   938   605  380.00000   60   43          -  380.00000      -  8109 1316s
   940   606  380.00000   20   24          -  380.00000      -  8092 1332s
   941   607  870.00000   76  243          -  380.00000      -  8084 1338s
   942   608  380.00000   36   30          -  380.00000      -  8075 1359s
   943   608  380.00000   38  335          -  380.00000      -  8066 1368s
   944   609  760.00000   59   26          -  380.00000      -  8058 1383s
   946   610  380.00000   27   24          -  380.00000      -  8041 1436s
   947   614  380.00000   10  283          -  380.00000      -   492 1448s
   949   615  380.00000   11  457          -  380.00000      -   513 1456s
   951   617  430.00000   11  295          -  380.00000      -   527 1467s
   953   618  380.00000   12  380          -  380.00000      -   555 1478s
   955   619  380.00000   12  507          -  380.00000      -   572 1486s
   957   621  380.00000  

    16    18  420.00000    5  766          -  420.00000      - 53037 1821s
    18    25  420.00000    5 1036          -  420.00000      - 73637 2040s
    25    46  420.00000    7  756          -  420.00000      - 68279 2626s
    46    99  420.00000   10  104          -  420.00000      - 62901 3000s

Cutting planes:
  Gomory: 1
  Cover: 179
  Implied bound: 36
  Clique: 6
  MIR: 56
  StrongCG: 1
  Zero half: 6
  RLT: 74
  Relax-and-lift: 5

Explored 107 nodes (5406242 simplex iterations) in 2000.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 4.200000000000e+02, gap -
No feasible solution found after 3000s.
Freeing default Gurobi environment



*** Solving instance of size: 50  ***

Using license file /Users/andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 3.3126039505004883 seconds.
Lower bounds found: |S| >= 2 |K| >= 2 |E| >= 6
Model ready
Changed value of parameter VarBranch to

  835131    2.6249455e+03   3.280899e+01   5.003117e+10    495s
  841686    2.5409345e+03   3.191829e+01   1.079020e+11    500s
  848011    2.5365735e+03   3.129855e+01   5.368746e+10    505s
  853876    2.5303235e+03   3.105565e+01   3.002275e+10    510s
  859626    2.5235946e+03   3.092482e+01   4.490315e+10    515s
  865491    2.5103857e+03   3.066955e+01   1.167928e+11    520s
  871356    2.4964645e+03   3.028092e+01   8.623416e+10    525s
  877566    2.4745197e+03   2.980272e+01   9.054390e+10    530s
  882971    2.4594919e+03   2.950842e+01   3.944312e+10    535s
  888721    2.4511408e+03   2.934808e+01   4.503101e+10    540s
  894701    2.4321370e+03   2.903315e+01   6.475882e+10    545s
  900796    2.4035265e+03   2.846188e+01   3.701803e+10    550s
  906891    2.3803694e+03   2.806426e+01   3.388776e+10    555s
  913101    2.3003703e+03   2.659275e+01   6.229864e+10    560s
  919081    2.2868875e+03   2.616612e+01   7.017631e+10    565s
  924946    2.2771431e+03   2.585726e+01

If we want to just compute lower bounds, run:

In [8]:
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    print('*** Bounds for instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    

    
    Λ   = [ np.append(a, arange(a.size + b + 1, (b+1)*a.size + 1) ) for b,a in enumerate([(E+1)+s for s in S]) ]
    Λ_β = {λ: d.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d.sum()/λ <= β - P}

    F_2, T_1 = max(Λ_β.items(), key=lambda x: x[1]) 
    F_1 = next(a for a,b in enumerate(Λ) if F_2 in b) + 1
    F_3 = [k for k in arange(F_1, K.size * F_1 + 1) if F_2 <= Q * k].pop(0)
    print('Lower bounds found: |S| >=', F_1, '|K| >=',F_3, '|E| >=', F_2)
    print((F_3,F_2,F_1))
    
    print('\n\n')

*** Bounds for instance of size: 10  ***

Lower bounds found: |S| >= 1 |K| >= 2 |E| >= 2
(2, 2, 1)



*** Bounds for instance of size: 12  ***

Lower bounds found: |S| >= 1 |K| >= 2 |E| >= 2
(2, 2, 1)



*** Bounds for instance of size: 14  ***

Lower bounds found: |S| >= 1 |K| >= 2 |E| >= 2
(2, 2, 1)



*** Bounds for instance of size: 16  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 2
(1, 2, 1)



*** Bounds for instance of size: 18  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 2
(1, 2, 1)



*** Bounds for instance of size: 20  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 2
(1, 2, 1)



*** Bounds for instance of size: 25  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 2
(1, 2, 1)



*** Bounds for instance of size: 30  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 2
(1, 2, 1)



*** Bounds for instance of size: 40  ***

Lower bounds found: |S| >= 1 |K| >= 1 |E| >= 3
(1, 3, 1)



*** Bounds for instance of size: 50  ***

Lower bounds found: |S| >= 2 |K| >= 2 |E

---

## Experimental

<div class="alert alert-block alert-success">
Second test, with less constraints.
</div>

In [9]:
time.sleep(360)

In [10]:
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    print('*** Solving instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    
    '''Multigraph'''
    m = 2*n

    # Domain for vehicles.                         Graph size:    (2*n) * (n + 1) + (2*n-1) * n = 4*(n**2) + n
    dom_v  = [ (0,j)   for j in range(1,m+1) ] 
    dom_v += [ (j,m+1) for j in range(1,m+1) ]
    dom_v += [ (i,j)   for i in range(1,m+1) for j in range(1,m+1) if j not in [i,i-n]]

    # Domain for pollsters.                        Graph size:    (n-1)**2 + (n-1) + n = n**2
    dom_e  = [ (i,i+n) for i in range(  1,n+1) ]
    dom_e += [ (i,j)   for i in range(1+n,m+1) for j in range(1,n+1) if j!=i-n ]
    
    C_minus, C_plus, C_0, C_m, C = range(1,n+1), range(n+1,2*n+1),  range(0,n+1), range(1,2*n+2),  range(1,2*n+1)
    
    '''MIP model'''
    mo = Model()
    x, y, z, b, f, w, B, u = {}, {}, {}, {}, {}, {}, {}, {}
    
    # Pollster variables
    ## Nodes
    b = mo.addVars( C_minus, E, S, vtype = 'B', name ='b')             # **begining**    n * |S|*|E|
    f = mo.addVars( C_plus,  E, S, vtype = 'B', name ='f')             # **ending**      n * |S|*|E|
    w = mo.addVars( C_minus, E, S, vtype = 'B', name ='w')             # **break**       n * |S|*|E|
    ## Arcs
    x = mo.addVars( dom_e, E, S, vtype = 'B', name = 'x')              # **Walking paths**     n^2 * |S|*|E|
    # Vehicle variables
    y = mo.addVars( dom_v, K, S, vtype = 'B', name = 'y')              # **Vehicle-paths**     (4*n^2+n)*|K|*|S|
    z = mo.addVars( dom_v, E, S, vtype = 'B', name = 'z')              # **t-paths**           (4*n^2+n)*|E|*|S|
    # Time and days variables
    B = mo.addVars( C, vtype = 'C', name = 'B', ub  = β )              # **In-Out timing**     2*n
    u = mo.addVars( S, vtype = 'B', name = 'u', obj = κ_0)             # **Day**               |S|

    mo.update()
    
    deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
    deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
    mo.setAttr('ModelSense', GRB.MINIMIZE)
    mo.update()
    
    
    start = time.time()
    
    # x & z vars interaction
    # 1a - Exclusive attention:      sum x[i,i+n] = 1
    mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');
    # 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
    mo.addConstrs( (x.sum('*',i,e,s) == x[i,i+n,e,s] - b[i,e,s] for i in C_minus for s in S for e in E), name='R-1b')
    mo.addConstrs( (x.sum(i,'*',e,s) == x[i-n,i,e,s] - f[i,e,s] for i in C_plus for s in S for e in E), name='R-1c')
    # 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
    mo.addConstrs( (z.sum('*',i,e,s) <= 1.0 - x[i,i+n,e,s] + b[i,e,s] for i in C_minus for s in S for e in E), name='R-1d')
    mo.addConstrs( (z.sum(i,'*',e,s) <= 1.0 - x[i-n,i,e,s] + f[i,e,s] for i in C_plus for s in S for e in E), name='R-1e')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == b[i,e,s] for i in C_minus for s in S for e in E), name='R-1f')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == -f[i,e,s] for i in C_plus for s in S for e in E), name='R-1g')
    # 1h – One trip per day for each pollster
    mo.addConstrs( (z.sum(0,'*',e,s) <= u[s] for s in S for e in E), name='R-1h')
    

    # y & z vars interaction
    start = time.time()
    # 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
    mo.addConstrs( (y.sum(i,'*','*',s) == b.sum(i,'*',s) for i in C_minus for s in S), name='R-2a')
    mo.addConstrs( (y.sum(i,'*','*',s) == f.sum(i,'*',s) for i in C_plus for s in S), name='R-2b')
    # 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
    mo.addConstrs( (y.sum('*',i,k,s) == y.sum(i,'*',k,s) for i in C for k in K for s in S ), name='R-2c')
    mo.addConstrs( (y.sum('*',2*n+1,k,s) == y.sum(0,'*',k,s) for k in K for s in S ), name='R-2d')
    # 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
    mo.addConstrs( (y.sum(0,'*',k,s) <= u[s] for k in K for s in S ), name='R-2e')
    # 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
    mo.addConstrs( (z.sum(i,j,'*',s) <= Q*y.sum(i,j,'*',s) for (i,j) in dom_v for s in S ), name='R-2f')

    
    # B vars enforce connected paths
    M = 1e+4
    # 3a,b – Arriving marker:  B[j] >= B[i] + t[i,j] + sum_s w[i] P - M(1 - sum_s x[i,j])
    mo.addConstrs( (B[i+n] - B[i] - d[i] >= P*w.sum(i,'*') for i in C_minus ), name='R-3a')
    mo.addConstrs( (B[j] - B[i+n] - t[i-1,j-1] >= -M*(1.0 - x.sum(i+n,j,'*')) 
                   for i in C_minus for j in C_minus if j!=i ), name='R-3b')
    ## Trivial
    mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
    # 3c — Arrival after transport: B[j] >= B[i] + τ_{i,j} - M(1 - sum_{s,k} y[i,j] )
    mo.addConstrs( 
        (B[j] - B[i] + M*(1.0 - y.sum(i,j,'*')) >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)] 
         for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c')
    # 3d — First transportation: B[i] >= tau_{0,i} - M(1 - sum_{s,k} y[0,i]^{k,s} )
    mo.addConstrs( (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)] - β*(1.0 - y.sum(0,i,'*','*')) 
                   for i in C ), name='R-3d')

    # 3e — Arrival marks: B[2n+1]^{s} >= B[i] + tau_{i,2n+1} - M(1-y[i,2n+1])
    mo.addConstrs( ( β - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] - M*(1.0 - y.sum(i,2*n+1,'*','*'))
                   for i in C ), name='R-3e')
    
    # w vars interact
    # 4a — Breaks TW:   p0 sum_{e,s} w[i] <= B[i] + d[i] <= p1 + M( 1 - sum_s w[i] )
    mo.addConstrs( (ρ_0*w.sum(i,'*') - B[i] - d[i] <= 0.0 for i in C_minus ), name='R-4a0')
    mo.addConstrs( (B[i] + d[i] - ρ_1 - β*(1.0 - w.sum(i,'*')) <= 0.0 for i in C_minus ), name='R-4a1')
    # 4b – One break per pollster:     w[i] <=  x[i,j]
    mo.addConstrs( (w[i,e,s] <= x[i,i+n,e,s] for i in C_minus for e in E for s in S ), name='R-4b')
    # 5c — Mandatory breaks:   sum_i w[i] = sum_j z[0,j]
    mo.addConstrs( (w.sum('*',e,s) == z.sum(0,'*',e,s) for e in E for s in S ), name='R-4b')

    
    # More

    mo.addConstr( z.sum(0,'*',0,0) == 1 , name='R-5a')
    mo.addConstr( y.sum(0,'*',0,0) == 1 , name='R-5d')

    if E.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e-1,s) for e in E for s in S if e > 0 ), name='R-5b')

        if S.size > 1:
            mo.addConstrs( 
                (b.sum('*',e,s) <= n - quicksum(b[i,e-1,r] for i in C_minus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5k')
            mo.addConstrs( 
                (f.sum('*',e,s) <= n - quicksum(f[i,e-1,r] for i in C_plus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5l')

    if S.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e,s-1) for e in E for s in S if s > 0 ), name='R-5c')
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k,s-1) for k in K for s in S if s > 0 ), name='R-5f')
        mo.addConstrs( (b.sum(i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5g')
        mo.addConstrs( (f.sum(i,'*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5h')
        mo.addConstrs( (x.sum('*',i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5i')
        mo.addConstrs( (x.sum(i,'*','*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5j')
        mo.addConstrs( (u[s] <= u[s-1] for s in S if s > 0), name='R-5m')

    if K.size > 1:
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k-1,s) for k in K for s in S if k > 0 ), name='R-5e')

    end = time.time()
    print('Constraints took {} seconds.'.format(end-start))
    print('Model ready')

    mo.update()
    
    
    '''Optimization'''
    # Parameters
    mo.Params.MIPFocus = 1;    
    mo.Params.Heuristics = 0.33
    mo.Params.Cuts = 3;      #<- w/o finds some feasible solutions 
    mo.Params.Method = 2

    mo.Params.SimplexPricing = 3
    mo.Params.CutAggPasses = 12;    mo.Params.CutPasses = 12;   mo.Params.PrePasses = 8

    mo.Params.ImproveStartTime = 100


    mo.setParam('Presolve', 2)
    mo.Params.GURO_PAR_PREPROBE = 3
    mo.Params.PreSparsify = 1
    mo.Params.PrePasses = 500 # best solution so far w/ 500
    
    # First round
    mo.Params.TimeLimit = 1000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')
        
        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})
        
        
        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')
        
        Out_File = 'Instances/Results_'+ str(n) +'-1000-LM.xlsx'
        
        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)
        
        print('Solution stored in local folder.')
    
    # Second round
    mo.Params.TimeLimit = 2000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')

        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})


        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')

        Out_File = 'Instances/Results_'+ str(n) +'-2000-LM.xlsx'

        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)

        print('Solution stored in local folder.')
    else:
        print('No feasible solution found after 3000s.')
    
    
    disposeDefaultEnv()
    mo.dispose()
    del mo, x, y, z, b, f, w, B, u
    print('\n\n')

*** Solving instance of size: 10  ***

Using license file /Users/Andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 0.11070013046264648 seconds.
Model ready
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter Heuristics to 0.33
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Cuts to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter SimplexPricing to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter CutAggPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter CutPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter PrePasses to 8
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter ImproveStartTime to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Change

 13724  2870     cutoff   44       480.00000  340.00000  29.2%  91.3  254s
 13817  2927  340.00000   33   29  480.00000  340.00000  29.2%  91.5  266s
 14037  2945     cutoff   44       480.00000  340.00000  29.2%  91.7  279s
 14085  2970  340.00000   39   45  480.00000  340.00000  29.2%  92.1  293s
 14176  3015  366.81516   43   49  480.00000  340.00000  29.2%  92.1  307s
 14551  3024  408.49780   52   15  480.00000  340.00000  29.2%  91.6  323s
 14607  3053  340.00000   40   21  480.00000  340.00000  29.2%  91.5  338s
 14725  3147  340.00000   35   23  480.00000  340.00000  29.2%  91.3  354s
 15050  3168  381.84079   39   55  480.00000  340.00000  29.2%  90.9  371s
 15138  3184  372.31230   49   51  480.00000  340.00000  29.2%  90.9  389s
 15274  3228  340.00000   40   24  480.00000  340.00000  29.2%  90.9  407s
 15544  3233     cutoff   53       480.00000  340.00000  29.2%  90.4  424s
 15575  3251     cutoff   38       480.00000  340.00000  29.2%  90.4  444s
 15659  3243     cutoff  

   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter CutAggPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter CutPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter PrePasses to 8
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter ImproveStartTime to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter GURO_PAR_PREPROBE to 3
   Prev: 1  Min: 0  Max: 3  Default: 1
Changed value of parameter PreSparsify to 1
   Prev: -1  Min: -1  Max: 1  Default: -1
Changed value of parameter PrePasses to 500
   Prev: 8  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 2513 rows, 5450 columns and 26146 nonzeros
Model fi

  1364   710  540.71806   16   26  820.00000  340.00000  58.5%   138  174s
  1365   710  680.00000   20   25  820.00000  340.00000  58.5%   138  177s
  1366   711  480.00000   39   29  820.00000  340.00000  58.5%   138  181s
  1369   713  680.00000   26   23  820.00000  340.00000  58.5%   138  188s

Resetting heuristic parameters to focus on improving solution
(using Heuristics=0.5 and RINS=10)...

  1380   725  340.00000   15   67  820.00000  340.00000  58.5%   172  190s
  1404   741  340.00000   21   54  820.00000  340.00000  58.5%   170  197s
  1408   752  340.00000   19   71  820.00000  340.00000  58.5%   170  200s
  1424   774  340.00000   24   46  820.00000  340.00000  58.5%   170  205s
  1447   776  340.00000   24   51  820.00000  340.00000  58.5%   170  211s
  1465   817  340.00000   26   46  820.00000  340.00000  58.5%   170  220s
  1585   792 infeasible   37       820.00000  340.00000  58.5%   163  231s
  1651   823 infeasible   40       820.00000  340.00000  58.5%   158  243

 38879  9631 infeasible   59       820.00000  340.00000  58.5%  88.9 2013s
 39277  9717  480.00000   63   79  820.00000  340.00000  58.5%  88.6 2042s
 39727  9741 infeasible   69       820.00000  340.00000  58.5%  88.5 2072s
 39885  9866  340.00000   69   48  820.00000  340.00000  58.5%  88.4 2102s
 40636  9936  340.00000   29   45  820.00000  340.00000  58.5%  87.8 2132s
 40929 10052 infeasible   82       820.00000  340.00000  58.5%  87.6 2162s
 41588 10064  375.30195  133   70  820.00000  340.00000  58.5%  87.4 2191s
 41727 10118     cutoff   73       820.00000  340.00000  58.5%  87.3 2222s
 42016 10325  340.00000   66   52  820.00000  340.00000  58.5%  87.2 2252s
 43043 10384  340.00000   30   38  820.00000  340.00000  58.5%  86.5 2282s
 43227 10466 infeasible   45       820.00000  340.00000  58.5%  86.4 2314s
 43537 10586 infeasible   58       820.00000  340.00000  58.5%  86.3 2346s
 44081 10801 infeasible   72       820.00000  340.00000  58.5%  86.1 2377s
 45028 10865 infeasible  

...

     0     0  340.00000    0   98          -  340.00000      -     -  664s
     0     2  340.00000    0   98          -  340.00000      -     -  846s
     3     8  340.00000    2  282          -  340.00000      - 16633  851s
    11    16  340.00000    3  281          -  340.00000      -  6225  855s
    27    43  340.00000    7  166          -  340.00000      -  3598  860s
    42    86  340.00000   11  139          -  340.00000      -  2726  865s
    85   186  340.00000   16  149          -  340.00000      -  1962  877s
   191   506  340.00000   30  134          -  340.00000      -  1526  900s
   587   654  340.00000   54  114          -  340.00000      -   998  917s
   985   852  340.00000   89   89          -  340.00000      -   862  932s
  1346   853  340.00000   61   98          -  340.00000      -   792  980s

Explored 1347 nodes (1665582 simplex iterations) in 1000.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 0

Time limit reached
Best objective -, bes

 41808 18888  358.43494  109   92          -  340.00000      -   320 3000s

Cutting planes:
  Cover: 1482
  Implied bound: 1127
  MIR: 906
  Flow cover: 3301
  Inf proof: 3
  Zero half: 952
  RLT: 514
  Relax-and-lift: 1447

Explored 42051 nodes (13995392 simplex iterations) in 2000.68 seconds
Thread count was 8 (of 8 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 3.400000000000e+02, gap -
No feasible solution found after 3000s.
Freeing default Gurobi environment



*** Solving instance of size: 30  ***

Using license file /Users/Andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 1.5621519088745117 seconds.
Model ready
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter Heuristics to 0.33
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Cuts to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Method to 2
   Prev: 

    45    71  340.00000   12  197          -  340.00000      -  4086 1141s
    71   160  340.00000   20  192          -  340.00000      -  3854 1201s
   160   323  340.00000   37  182          -  340.00000      -  3692 1356s
   336   961  340.00000   66   77          -  340.00000      -  3273 1515s
  1364  1411  730.00000  136   64          -  340.00000      -  1770 1560s
  2549  1412  340.00000   73  117          -  340.00000      -  1166 1701s
  2551  1413  340.00000   90   44          -  340.00000      -  1165 1776s
  2552  1414  415.00000  181   30          -  340.00000      -  1164 1821s
  2553  1415  340.00000   62  171          -  340.00000      -  1164 1828s
  2554  1415  340.00000  138   23          -  340.00000      -  1163 1875s
  2555  1416  560.00000  191  303          -  340.00000      -  1163 1909s
  2556  1417  340.00000   52   27          -  340.00000      -  1162 1981s
  2557  1417  340.00000   82  358          -  340.00000      -  1162 2018s
  2558  1418  461.37508  

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 24007 rows, 91122 columns and 409854 nonzeros
Model fingerprint: 0xec489215
Variable types: 80 continuous, 91042 integer (91042 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 1e+04]
Presolved: 23960 rows, 91906 columns, 646934 nonzeros

Continuing optimization...

     2     4  340.00000    1  546          -  340.00000      - 106623 1302s
     4     8  340.00000    2  863          -  340.00000      - 66136 1439s
     7     8  340.00000    2  902          -  340.00000      - 48744 1441s
     8    12  340.00000    3  910          -  340.00000      - 47658 1989s
    12    16  340.00000    3  893          -  340.00000      - 45056 2206s
    16    20  340.00000    4  880          -  340.00000      - 44114 2259s
    20    24  340.00000    4  901          -  340.00000      - 39402 2383s
    24    28  340.

     2     4  340.00000    1 2979          -  340.00000      - 203438 2731s

Explored 3 nodes (869180 simplex iterations) in 2000.06 seconds
Thread count was 8 (of 8 available processors)

Solution count 0

Time limit reached
Best objective -, best bound 3.400000000000e+02, gap -
No feasible solution found after 3000s.
Freeing default Gurobi environment





<div class="alert alert-block alert-success">
Third test: solver default.
</div>

In [11]:
time.sleep(360)

In [12]:
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    print('*** Solving instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    
    '''Multigraph'''
    m = 2*n

    # Domain for vehicles.                         Graph size:    (2*n) * (n + 1) + (2*n-1) * n = 4*(n**2) + n
    dom_v  = [ (0,j)   for j in range(1,m+1) ] 
    dom_v += [ (j,m+1) for j in range(1,m+1) ]
    dom_v += [ (i,j)   for i in range(1,m+1) for j in range(1,m+1) if j not in [i,i-n]]

    # Domain for pollsters.                        Graph size:    (n-1)**2 + (n-1) + n = n**2
    dom_e  = [ (i,i+n) for i in range(  1,n+1) ]
    dom_e += [ (i,j)   for i in range(1+n,m+1) for j in range(1,n+1) if j!=i-n ]
    
    C_minus, C_plus, C_0, C_m, C = range(1,n+1), range(n+1,2*n+1),  range(0,n+1), range(1,2*n+2),  range(1,2*n+1)
    
    '''MIP model'''
    mo = Model()
    x, y, z, b, f, w, B, u = {}, {}, {}, {}, {}, {}, {}, {}
    
    # Pollster variables
    ## Nodes
    b = mo.addVars( C_minus, E, S, vtype = 'B', name ='b')             # **begining**    n * |S|*|E|
    f = mo.addVars( C_plus,  E, S, vtype = 'B', name ='f')             # **ending**      n * |S|*|E|
    w = mo.addVars( C_minus, E, S, vtype = 'B', name ='w')             # **break**       n * |S|*|E|
    ## Arcs
    x = mo.addVars( dom_e, E, S, vtype = 'B', name = 'x')              # **Walking paths**     n^2 * |S|*|E|
    # Vehicle variables
    y = mo.addVars( dom_v, K, S, vtype = 'B', name = 'y')              # **Vehicle-paths**     (4*n^2+n)*|K|*|S|
    z = mo.addVars( dom_v, E, S, vtype = 'B', name = 'z')              # **t-paths**           (4*n^2+n)*|E|*|S|
    # Time and days variables
    B = mo.addVars( C, vtype = 'C', name = 'B', ub  = β )              # **In-Out timing**     2*n
    u = mo.addVars( S, vtype = 'B', name = 'u', obj = κ_0)             # **Day**               |S|

    mo.update()
    
    deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
    deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
    mo.setAttr('ModelSense', GRB.MINIMIZE)
    mo.update()
    
    
    start = time.time()
    
    # x & z vars interaction
    # 1a - Exclusive attention:      sum x[i,i+n] = 1
    mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');
    # 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
    mo.addConstrs( (x.sum('*',i,e,s) == x[i,i+n,e,s] - b[i,e,s] for i in C_minus for s in S for e in E), name='R-1b')
    mo.addConstrs( (x.sum(i,'*',e,s) == x[i-n,i,e,s] - f[i,e,s] for i in C_plus for s in S for e in E), name='R-1c')
    # 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
    mo.addConstrs( (z.sum('*',i,e,s) <= 1.0 - x[i,i+n,e,s] + b[i,e,s] for i in C_minus for s in S for e in E), name='R-1d')
    mo.addConstrs( (z.sum(i,'*',e,s) <= 1.0 - x[i-n,i,e,s] + f[i,e,s] for i in C_plus for s in S for e in E), name='R-1e')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == b[i,e,s] for i in C_minus for s in S for e in E), name='R-1f')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == -f[i,e,s] for i in C_plus for s in S for e in E), name='R-1g')
    # 1h – One trip per day for each pollster
    mo.addConstrs( (z.sum(0,'*',e,s) <= u[s] for s in S for e in E), name='R-1h')
    

    # y & z vars interaction
    start = time.time()
    # 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
    mo.addConstrs( (y.sum(i,'*','*',s) == b.sum(i,'*',s) for i in C_minus for s in S), name='R-2a')
    mo.addConstrs( (y.sum(i,'*','*',s) == f.sum(i,'*',s) for i in C_plus for s in S), name='R-2b')
    # 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
    mo.addConstrs( (y.sum('*',i,k,s) == y.sum(i,'*',k,s) for i in C for k in K for s in S ), name='R-2c')
    mo.addConstrs( (y.sum('*',2*n+1,k,s) == y.sum(0,'*',k,s) for k in K for s in S ), name='R-2d')
    # 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
    mo.addConstrs( (y.sum(0,'*',k,s) <= u[s] for k in K for s in S ), name='R-2e')
    # 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
    mo.addConstrs( (z.sum(i,j,'*',s) <= Q*y.sum(i,j,'*',s) for (i,j) in dom_v for s in S ), name='R-2f')

    
    # B vars enforce connected paths
    M = 1e+4
    # 3a,b – Arriving marker:  B[j] >= B[i] + t[i,j] + sum_s w[i] P - M(1 - sum_s x[i,j])
    mo.addConstrs( (B[i+n] - B[i] - d[i] >= P*w.sum(i,'*') for i in C_minus ), name='R-3a')
    mo.addConstrs( (B[j] - B[i+n] - t[i-1,j-1] >= -M*(1.0 - x.sum(i+n,j,'*')) 
                   for i in C_minus for j in C_minus if j!=i ), name='R-3b')
    ## Trivial
    mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
    # 3c — Arrival after transport: B[j] >= B[i] + τ_{i,j} - M(1 - sum_{s,k} y[i,j] )
    mo.addConstrs( 
        (B[j] - B[i] + M*(1.0 - y.sum(i,j,'*')) >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)] 
         for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c')
    # 3d — First transportation: B[i] >= tau_{0,i} - M(1 - sum_{s,k} y[0,i]^{k,s} )
    mo.addConstrs( (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)] - β*(1.0 - y.sum(0,i,'*','*')) 
                   for i in C ), name='R-3d')

    # 3e — Arrival marks: B[2n+1]^{s} >= B[i] + tau_{i,2n+1} - M(1-y[i,2n+1])
    mo.addConstrs( ( β * u[s] - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] - M*(1.0 - y.sum(i,2*n+1,'*',s))
                   for i in C for s in S ), name='R-3e')
    
    # w vars interact
    # 4a — Breaks TW:   p0 sum_{e,s} w[i] <= B[i] + d[i] <= p1 + M( 1 - sum_s w[i] )
    mo.addConstrs( (ρ_0*w.sum(i,'*') - B[i] - d[i] <= 0.0 for i in C_minus ), name='R-4a0')
    mo.addConstrs( (B[i] + d[i] - ρ_1 - β*(1.0 - w.sum(i,'*')) <= 0.0 for i in C_minus ), name='R-4a1')
    # 4b – One break per pollster:     w[i] <=  x[i,j]
    mo.addConstrs( (w[i,e,s] <= x[i,i+n,e,s] for i in C_minus for e in E for s in S ), name='R-4b')
    # 5c — Mandatory breaks:   sum_i w[i] = sum_j z[0,j]
    mo.addConstrs( (w.sum('*',e,s) == z.sum(0,'*',e,s) for e in E for s in S ), name='R-4b')

    
    # More

    mo.addConstr( z.sum(0,'*',0,0) == 1 , name='R-5a')
    mo.addConstr( y.sum(0,'*',0,0) == 1 , name='R-5d')

    if E.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e-1,s) for e in E for s in S if e > 0 ), name='R-5b')

        if S.size > 1:
            mo.addConstrs( 
                (b.sum('*',e,s) <= n - quicksum(b[i,e-1,r] for i in C_minus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5k')
            mo.addConstrs( 
                (f.sum('*',e,s) <= n - quicksum(f[i,e-1,r] for i in C_plus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5l')

    if S.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e,s-1) for e in E for s in S if s > 0 ), name='R-5c')
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k,s-1) for k in K for s in S if s > 0 ), name='R-5f')
        mo.addConstrs( (b.sum(i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5g')
        mo.addConstrs( (f.sum(i,'*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5h')
        mo.addConstrs( (x.sum('*',i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5i')
        mo.addConstrs( (x.sum(i,'*','*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5j')
        mo.addConstrs( (u[s] <= u[s-1] for s in S if s > 0), name='R-5m')

    if K.size > 1:
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k-1,s) for k in K for s in S if k > 0 ), name='R-5e')

    end = time.time()
    print('Constraints took {} seconds.'.format(end-start))
    print('Model ready')

    mo.update()
    
    
    '''Optimization'''
    # First round
    mo.Params.TimeLimit = 1000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')
        
        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})
        
        
        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')
        
        Out_File = 'Instances/Results_'+ str(n) +'-1000-G.xlsx'
        
        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)
        
        print('Solution stored in local folder.')
    
    # Second round
    mo.Params.TimeLimit = 2000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')

        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})


        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')

        Out_File = 'Instances/Results_'+ str(n) +'-2000-G.xlsx'

        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)

        print('Solution stored in local folder.')
    else:
        print('No feasible solution found after 3000s.')
    
    
    disposeDefaultEnv()
    mo.dispose()
    del mo, x, y, z, b, f, w, B, u
    print('\n\n')

*** Solving instance of size: 10  ***

Using license file /Users/Andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 0.11303400993347168 seconds.
Model ready
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1268 rows, 2871 columns and 13606 nonzeros
Model fingerprint: 0xd24b4beb
Variable types: 20 continuous, 2851 integer (2851 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+04]
Presolve removed 14 rows and 1 columns
Presolve time: 0.03s
Presolved: 1254 rows, 2870 columns, 13400 nonzeros
Variable types: 20 continuous, 2850 integer (2850 binary)

Root relaxation: objective 3.400000e+02, 1934 iterations, 0.11 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

 131153 16163  426.83209   61   58  480.00000  340.00000  29.2%   114  442s
 132622 16163  340.15515   84   25  480.00000  340.00000  29.2%   114  445s
 134984 16372  340.00000   86   47  480.00000  340.00000  29.2%   114  453s
 136264 16433 infeasible   84       480.00000  340.00000  29.2%   114  457s
 137762 16528  340.00000  102   20  480.00000  340.00000  29.2%   114  460s
 140144 16657 infeasible   88       480.00000  340.00000  29.2%   113  468s
 141432 16687 infeasible   72       480.00000  340.00000  29.2%   114  471s
 141985 16730  340.00000  105   37  480.00000  340.00000  29.2%   113  475s
 145237 16900 infeasible   77       480.00000  340.00000  29.2%   113  482s
 145908 17055     cutoff  102       480.00000  340.00000  29.2%   113  486s
 149149 17145  340.00000   71   21  480.00000  340.00000  29.2%   113  493s
 150755 17277  340.00000   92   24  480.00000  340.00000  29.2%   113  496s
 153174 17501  363.44604   61   39  480.00000  340.00000  29.2%   112  503s
 154239 1761


Cutting planes:
  Gomory: 1
  Cover: 1
  Implied bound: 4
  Projected implied bound: 1
  MIR: 5
  StrongCG: 1
  Flow cover: 9
  GUB cover: 1
  RLT: 8
  Relax-and-lift: 6

Explored 822180 nodes (33146553 simplex iterations) in 1000.04 seconds
Thread count was 8 (of 8 available processors)

Solution count 3: 480 580 620 

Time limit reached
Best objective 4.800000000000e+02, best bound 3.400000000000e+02, gap 29.1667%
A solution was found after 1000.0519652366638s with 2 pollsters, 2 vehicles, and 1 days.
Solution stored in local folder.
Changed value of parameter TimeLimit to 2000.0
   Prev: 1000.0  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1268 rows, 2871 columns and 13606 nonzeros
Model fingerprint: 0xd24b4beb
Variable types: 20 continuous, 2851 integer (2851 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00,

 1738450 95293 infeasible   68       480.00000  340.00000  29.2%  28.2 1465s
 1746520 96061  340.00000   60   27  480.00000  340.00000  29.2%  28.2 1470s
 1756242 96454 infeasible   74       480.00000  340.00000  29.2%  28.1 1475s
 1766407 96971 infeasible   65       480.00000  340.00000  29.2%  28.1 1480s
 1773781 97740  340.00000   53   35  480.00000  340.00000  29.2%  28.1 1485s
 1785513 98384 infeasible   59       480.00000  340.00000  29.2%  28.0 1490s
 1794237 98727  340.00000   55   37  480.00000  340.00000  29.2%  27.9 1495s
 1806153 99359 infeasible   54       480.00000  340.00000  29.2%  27.9 1500s
 1814477 100101 infeasible   61       480.00000  340.00000  29.2%  27.8 1505s
 1824419 100965  340.00000   56   41  480.00000  340.00000  29.2%  27.8 1510s
 1831984 101664  340.00000   57   24  480.00000  340.00000  29.2%  27.7 1515s
 1844311 102284 infeasible   58       480.00000  340.00000  29.2%  27.7 1520s
 1854238 102591  340.77570   53   43  480.00000  340.00000  29.2%  27.6 

 2736931 132298  340.00000   60   34  480.00000  340.00000  29.2%  24.4 1995s
 2746977 132369  376.98995   75   25  480.00000  340.00000  29.2%  24.4 2000s
 2754121 132492 infeasible   76       480.00000  340.00000  29.2%  24.4 2005s
 2765314 132637  340.00000   53   47  480.00000  340.00000  29.2%  24.3 2010s
 2776370 132990 infeasible   61       480.00000  340.00000  29.2%  24.3 2015s
 2785327 133329  340.41678   65   60  480.00000  340.00000  29.2%  24.3 2020s
 2794486 133741 infeasible   64       480.00000  340.00000  29.2%  24.3 2025s
 2804708 134026  440.00000   72   27  480.00000  340.00000  29.2%  24.2 2030s
 2815110 134505 infeasible   61       480.00000  340.00000  29.2%  24.2 2035s
 2823117 134884 infeasible   63       480.00000  340.00000  29.2%  24.2 2040s
 2834220 135429  340.00000   60   38  480.00000  340.00000  29.2%  24.2 2045s
 2844414 135571     cutoff   69       480.00000  340.00000  29.2%  24.1 2050s
 2852834 136084  340.00000   60   39  480.00000  340.00000  29.2

...

  1224   926  340.00000   47   58          -  340.00000      -  2129  192s
  1226   927  340.00000   19   27          -  340.00000      -  2126  199s
  1227   928  340.00000  111  249          -  340.00000      -  2124  201s
  1228   929  340.00000   23   41          -  340.00000      -  2123  205s
  1230   930  660.00000  158   39          -  340.00000      -  2119  216s
  1231   934  340.00000   10  258          -  340.00000      -   217  231s
  1233   937  340.00000   11  313          -  340.00000      -   226  246s
  1237   940  340.00000   12  354          -  340.00000      -   236  259s
  1241   942  340.00000   12  378          -  340.00000      -   247  274s
  1245   945  340.00000   13  307          -  340.00000      -   261  282s
  1249   948  340.00000   13  313          -  340.00000      -   273  288s
  1253   950  340.00000   14  299          -  340.00000      -   283  292s
  1257   953  340.00000   14  304          -  340.00000      -   295  296s
  1261   959  340.00000  

  1764  1251  340.00000   50   28          -  340.00000      -  1028  295s
  1765  1252  340.00000   93  272          -  340.00000      -  1028  300s
  1766  1253  340.00000   89   24          -  340.00000      -  1027  311s
  1767  1253  340.00000   60  280          -  340.00000      -  1026  317s
  1768  1254  340.00000   93   12          -  340.00000      -  1026  335s
  1770  1255  480.00000  110   26          -  340.00000      -  1025  342s
  1771  1256  340.00000   83  305          -  340.00000      -  1024  348s
  1772  1257  340.00000   43   39          -  340.00000      -  1023  358s
  1773  1257  340.00000   17  258          -  340.00000      -  1023  362s
  1774  1258  430.00000  116   28          -  340.00000      -  1022  370s
  1775  1259  340.00000   75  336          -  340.00000      -  1022  375s
  1776  1259  340.00000   50   39          -  340.00000      -  1021  386s
  1777  1260  340.00000   77  173          -  340.00000      -  1021  390s
  1778  1261  340.00000  

Variable types: 60 continuous, 33007 integer (33007 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   21905    5.3533806e+02   0.000000e+00   2.951214e+06      5s
   25845    5.1271975e+02   0.000000e+00   1.191605e+06     10s
   29855    5.0856075e+02   0.000000e+00   2.246916e+06     15s
Concurrent spin time: 4.57s

Solved with dual simplex

Root relaxation: objective 3.400000e+02, 16347 iterations, 17.41 seconds
Total elapsed time = 22.26s
Total elapsed time = 25.10s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  340.00000    0   39          -  340.00000      -     -   27s
     0     0  340.00000    0  650          -  340.00000      -     -   54s
     0     0  340.00000    0  590          -  340.00000      -     -   75s
     0    

   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 24087 rows, 91122 columns and 410094 nonzeros
Model fingerprint: 0xdedd6b5b
Variable types: 80 continuous, 91042 integer (91042 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 1e+04]
Presolve added 0 rows and 9 columns
Presolve removed 47 rows and 0 columns
Presolve time: 0.92s
Presolved: 24040 rows, 91131 columns, 406132 nonzeros
Variable types: 80 continuous, 91051 integer (91051 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   25201    3.4000000e+02   6.525584e+02   1.769604e+10      5s
   48813    3.4000000e+02   6.378891e+02   2.336690e+10     10s
   69887    3.4107204e+02   6.175797e+02   3.689521e+10     15s
 

  332328    3.4000000e+02   2.495002e+02   1.160901e+11    135s
  341571    3.4000000e+02   2.479346e+02   5.314596e+10    140s
  351086    3.4000000e+02   2.285411e+02   4.832571e+10    145s
  358216    3.4000000e+02   2.263454e+02   6.248181e+10    150s
  365354    3.4000000e+02   2.253763e+02   3.079308e+10    155s
  372262    3.4000000e+02   2.092856e+02   7.598171e+10    160s
  381312    3.4000000e+02   2.074578e+02   2.830864e+10    165s
  390315    3.4000000e+02   1.934872e+02   4.702196e+10    170s
  399141    3.4000000e+02   1.915841e+02   3.403918e+10    175s
  408275    3.4000000e+02   1.744556e+02   3.579996e+10    180s
  418486    3.4000000e+02   1.699571e+02   3.649747e+10    185s
  428019    3.4000000e+02   1.310380e+02   8.531881e+10    190s
  437663    3.4000000e+02   1.249182e+02   3.333448e+11    195s
  443873    3.4000000e+02   1.238165e+02   3.133938e+11    200s
  451578    3.4000000e+02   1.232977e+02   2.147148e+11    205s
  457328    3.4000000e+02   1.226893e+02

<div class="alert alert-block alert-success">
Fourth test: solver default LM with logical constraints.
</div>

In [13]:
time.sleep(360)

In [14]:
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    print('*** Solving instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    
    '''Multigraph'''
    m = 2*n

    # Domain for vehicles.                         Graph size:    (2*n) * (n + 1) + (2*n-1) * n = 4*(n**2) + n
    dom_v  = [ (0,j)   for j in range(1,m+1) ] 
    dom_v += [ (j,m+1) for j in range(1,m+1) ]
    dom_v += [ (i,j)   for i in range(1,m+1) for j in range(1,m+1) if j not in [i,i-n]]

    # Domain for pollsters.                        Graph size:    (n-1)**2 + (n-1) + n = n**2
    dom_e  = [ (i,i+n) for i in range(  1,n+1) ]
    dom_e += [ (i,j)   for i in range(1+n,m+1) for j in range(1,n+1) if j!=i-n ]
    
    C_minus, C_plus, C_0, C_m, C = range(1,n+1), range(n+1,2*n+1),  range(0,n+1), range(1,2*n+2),  range(1,2*n+1)
    
    '''MIP model'''
    mo = Model()
    x, y, z, b, f, w, B, u, wes, xes, yks = {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}
    
    # Pollster variables
    ## Nodes
    b = mo.addVars( C_minus, E, S, vtype = 'B', name ='b')             # **begining**    n * |S|*|E|
    f = mo.addVars( C_plus,  E, S, vtype = 'B', name ='f')             # **ending**      n * |S|*|E|
    w = mo.addVars( C_minus, E, S, vtype = 'B', name ='w')             # **break**       n * |S|*|E|
    ## Arcs
    x = mo.addVars( dom_e, E, S, vtype = 'B', name = 'x')              # **Walking paths**     n^2 * |S|*|E|
    # Vehicle variables
    y = mo.addVars( dom_v, K, S, vtype = 'B', name = 'y')              # **Vehicle-paths**     (4*n^2+n)*|K|*|S|
    z = mo.addVars( dom_v, E, S, vtype = 'B', name = 'z')              # **t-paths**           (4*n^2+n)*|E|*|S|
    # Time and days variables
    B = mo.addVars( C, vtype = 'C', name = 'B', ub  = β )              # **In-Out timing**     2*n
    u = mo.addVars( S, vtype = 'B', name = 'u', obj = κ_0)             # **Day**               |S|

    wes = mo.addVars( C_minus, vtype = 'B', name ='wes')             # **break sum**       n
    xes = mo.addVars( ((i,j) for i in C_plus for j in C_minus if j!=i), vtype = 'B', name = 'xes')     #      n^2
    yks = mo.addVars( dom_v, vtype = 'B', name = 'yks')                                                #     4*n^2+n
    
    mo.update()
    
    deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
    deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
    mo.setAttr('ModelSense', GRB.MINIMIZE)
    mo.update()
    
    
    start = time.time()
    
    # x & z vars interaction
    # 1a - Exclusive attention:      sum x[i,i+n] = 1
    mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');
    # 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
    mo.addConstrs( (x.sum('*',i,e,s) == x[i,i+n,e,s] - b[i,e,s] for i in C_minus for s in S for e in E), name='R-1b')
    mo.addConstrs( (x.sum(i,'*',e,s) == x[i-n,i,e,s] - f[i,e,s] for i in C_plus for s in S for e in E), name='R-1c')
    # 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
    mo.addConstrs( (z.sum('*',i,e,s) <= 1.0 - x[i,i+n,e,s] + b[i,e,s] for i in C_minus for s in S for e in E), name='R-1d')
    mo.addConstrs( (z.sum(i,'*',e,s) <= 1.0 - x[i-n,i,e,s] + f[i,e,s] for i in C_plus for s in S for e in E), name='R-1e')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == b[i,e,s] for i in C_minus for s in S for e in E), name='R-1f')
    mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == -f[i,e,s] for i in C_plus for s in S for e in E), name='R-1g')
    # 1h – One trip per day for each pollster
    mo.addConstrs( (z.sum(0,'*',e,s) <= u[s] for s in S for e in E), name='R-1h')
    

    # y & z vars interaction
    start = time.time()
    # 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
    mo.addConstrs( (y.sum(i,'*','*',s) == b.sum(i,'*',s) for i in C_minus for s in S), name='R-2a')
    mo.addConstrs( (y.sum(i,'*','*',s) == f.sum(i,'*',s) for i in C_plus for s in S), name='R-2b')
    # 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
    mo.addConstrs( (y.sum('*',i,k,s) == y.sum(i,'*',k,s) for i in C for k in K for s in S ), name='R-2c')
    mo.addConstrs( (y.sum('*',2*n+1,k,s) == y.sum(0,'*',k,s) for k in K for s in S ), name='R-2d')
    # 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
    mo.addConstrs( (y.sum(0,'*',k,s) <= u[s] for k in K for s in S ), name='R-2e')
    # 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
    mo.addConstrs( (z.sum(i,j,'*',s) <= Q*y.sum(i,j,'*',s) for (i,j) in dom_v for s in S ), name='R-2f')

    
    # B vars enforce connected paths
    # 3x,y - Indicator variables over taken arcs
    mo.addConstrs( (x.sum(i,j,'*') == xes[i,j] for i in C_plus for j in C_minus if j!=i), name='R-3x' )
    mo.addConstrs( (y.sum(i,j,'*') == yks[i,j] for (i,j) in dom_v), name='R-3y' )
    # 3a,b – Arriving marker:
    mo.addConstrs( (B[i+n] - B[i] - d[i] >= P * wes[i] for i in C_minus ), name='R-3a')
    mo.addConstrs( ( (xes[i+n,j] == 1.0) >> (B[j] - B[i+n] - t[i-1,j-1] >= 0.0) 
                      for i in C_minus for j in C_minus if j!=i), name='R-3b');
    ## Trivial
    mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
    # 3c — Arrival after transport:
    mo.addConstrs( ( (yks[i,j] == 1.0) >> (B[j] - B[i] 
                                       >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)]) 
                      for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c');

    # 3d — First transportation:
    mo.addConstrs( ( (yks[0,i] == 1.0) >> (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)]) for i in C ), name='R-3d');
    # 3e — Arrival marks:
    mo.addConstrs( ( (yks[i,2*n+1] == 1.0) >> ( β - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] )
                      for i in C ), name='R-3e');
    

    # w vars interact
    # 4w - Breaks sum
    mo.addConstrs( (w.sum(i,'*') == wes[i] for i in C_minus), name='R-4w' )
    # 4a — Breaks TW:
    mo.addConstrs( ( (wes[i] == 1.0) >> (ρ_0 - B[i] - d[i] <= 0.0) for i in C_minus ), name='R-4a0')
    mo.addConstrs( ( (wes[i] == 1.0) >> (B[i] + d[i] <= ρ_1) for i in C_minus ), name='R-4a1')
    # 4b – One break per pollster:
    mo.addConstrs( (w[i,e,s] <= x[i,i+n,e,s] for i in C_minus for e in E for s in S ), name='R-4b')
    # 5c — Mandatory breaks:
    mo.addConstrs( (w.sum('*',e,s) == z.sum(0,'*',e,s) for e in E for s in S ), name='R-4b')

    mo.update()

    
    # More

    mo.addConstr( z.sum(0,'*',0,0) == 1 , name='R-5a')
    mo.addConstr( y.sum(0,'*',0,0) == 1 , name='R-5d')

    if E.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e-1,s) for e in E for s in S if e > 0 ), name='R-5b')

        if S.size > 1:
            mo.addConstrs( 
                (b.sum('*',e,s) <= n - quicksum(b[i,e-1,r] for i in C_minus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5k')
            mo.addConstrs( 
                (f.sum('*',e,s) <= n - quicksum(f[i,e-1,r] for i in C_plus for r in S if r<=s-1) 
                 for e in E for s in S if e > 0 if s > 0 ), name='R-5l')

    if S.size > 1:
        mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e,s-1) for e in E for s in S if s > 0 ), name='R-5c')
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k,s-1) for k in K for s in S if s > 0 ), name='R-5f')
        mo.addConstrs( (b.sum(i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5g')
        mo.addConstrs( (f.sum(i,'*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5h')
        mo.addConstrs( (x.sum('*',i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5i')
        mo.addConstrs( (x.sum(i,'*','*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5j')
        mo.addConstrs( (u[s] <= u[s-1] for s in S if s > 0), name='R-5m')

    if K.size > 1:
        mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k-1,s) for k in K for s in S if k > 0 ), name='R-5e')

    end = time.time()
    print('Constraints took {} seconds.'.format(end-start))
    print('Model ready')

    mo.update()
    
    
    '''Optimization'''
    # Parameters
    
    # First round
    mo.Params.TimeLimit = 1000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')
        
        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})
        
        
        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')
        
        Out_File = 'Instances/Results_'+ str(n) +'-1000-LM.xlsx'
        
        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)
        
        print('Solution stored in local folder.')
    
    # Second round
    mo.Params.TimeLimit = 2000
    mo.optimize()
    
    if hasattr(u[0], 'x'):
        #print('Feasible solution found after 1000s.')

        X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
        Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
        Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

        A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
        F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
        W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
        U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})


        Active_Days      = U.keys()
        Active_Pollsters = {v[1] for v in A.keys()}
        Active_Vehicles  = {v[2] for v in Y.keys()}
        print('A solution was found after ' + str(mo.RunTime) + 's with',len(Active_Pollsters),'pollsters,',
              len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')

        Out_File = 'Instances/Results_'+ str(n) +'-2000-LM.xlsx'

        with pd.ExcelWriter(Out_File) as writer:
            # General results
            Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP'], 
                              '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %']})
            Hoja.to_excel(writer, 'Summary', header=False, index= False)
            writer.sheets['Summary'].set_column('A:A', 15)

            # Pollster routing
            X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                X_day = (v[:-1] for v in X.keys() if v[-1] == days)
                for pairs in X_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
                    X_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Pollster routing')

            # Vehicle routing
            Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
            for days in Active_Days:
                Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
                for pairs in Y_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    Y_visits[days][coords] = pairs[-1] + 1

            Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Vehicle routing')

            # Shared routing
            Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
            for days in Active_Days:
                Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
                for pairs in Z_day:
                    coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
                    if Z_visits[days][coords] == '':
                        Z_visits[days][coords] = str(pairs[-1] + 1)
                    else:
                        Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)

            Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
            Hoja.to_excel(writer, 'Shared routing')

            # Times and breaks
            W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
            for days in Active_Days:
                for v in (v[:-1] for v in W.keys() if v[-1] == days):
                    W_breaks[days][v[0]-1] = v[1]+1

            Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
            Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
            Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
            Hoja.index = arange(1,n+1)
            Hoja.to_excel(writer, 'Times and breaks per pollster')
            writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)

        print('Solution stored in local folder.')
    else:
        print('No feasible solution found after 3000s.')
    
    
    disposeDefaultEnv()
    mo.dispose()
    del mo, x, y, z, b, f, w, B, u, wes, xes, yks
    print('\n\n')

*** Solving instance of size: 10  ***

Using license file /Users/Andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 0.11726593971252441 seconds.
Model ready
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1268 rows, 3391 columns and 13076 nonzeros
Model fingerprint: 0x532df7bf
Model has 520 general constraints
Variable types: 20 continuous, 3371 integer (3371 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+01]
Presolve added 470 rows and 0 columns
Presolve removed 0 rows and 37 columns
Presolve time: 0.06s
Presolved: 1738 rows, 3354 columns, 14292 nonzeros
Variable types: 20 continuous, 3334 integer (3334 binary)

Root relaxation: objective 3.400000e+02, 3813 iterations, 0.25 seconds

    Nodes    |    Current Node    |   

 85717  8647  340.00000   50   59  480.00000  340.00000  29.2%   200  542s
 86774  8741     cutoff   47       480.00000  340.00000  29.2%   200  550s
 88017  8861     cutoff   49       480.00000  340.00000  29.2%   200  557s
 89366  8885     cutoff   58       480.00000  340.00000  29.2%   200  568s
 90505  8965  343.96632   53   72  480.00000  340.00000  29.2%   200  576s
 92029  9083  340.00000   51   63  480.00000  340.00000  29.2%   200  583s
 93152  9196 infeasible   42       480.00000  340.00000  29.2%   200  590s
 94549  9268  374.39489   51  103  480.00000  340.00000  29.2%   200  598s
 95738  9432  362.78322   53   84  480.00000  340.00000  29.2%   200  604s
 97597  9533  340.00000   37   84  480.00000  340.00000  29.2%   199  611s
 99189  9550     cutoff   62       480.00000  340.00000  29.2%   199  617s
 100515  9613  340.00000   42   69  480.00000  340.00000  29.2%   199  623s
 101828  9694  340.00000   34   57  480.00000  340.00000  29.2%   199  630s
 103128  9702  403.8138

 199543 16046  368.97427   51   78  480.00000  340.00000  29.2%   193 1147s
 200815 16162  388.62534   63   66  480.00000  340.00000  29.2%   193 1152s
 202415 16230  389.11247   48   80  480.00000  340.00000  29.2%   193 1158s
 203975 16282     cutoff   49       480.00000  340.00000  29.2%   192 1164s
 205153 16354  383.70628   56   41  480.00000  340.00000  29.2%   192 1169s
 206233 16386     cutoff   72       480.00000  340.00000  29.2%   192 1175s
 207636 16433 infeasible   55       480.00000  340.00000  29.2%   191 1180s
 209217 16476  390.06248   55   49  480.00000  340.00000  29.2%   191 1186s
 210482 16525  390.63333   65   81  480.00000  340.00000  29.2%   191 1191s
 211674 16522     cutoff   52       480.00000  340.00000  29.2%   190 1197s
 212889 16568  341.22364   67   97  480.00000  340.00000  29.2%   190 1202s
 213856 16628  340.00000   40   57  480.00000  340.00000  29.2%   190 1208s
 214973 16695     cutoff   66       480.00000  340.00000  29.2%   190 1213s
 216037 1677

 330551 25206  353.05312   63   48  480.00000  340.00000  29.2%   183 1826s
 331392 25205  357.41275   51   61  480.00000  340.00000  29.2%   183 1831s
 332789 25349  340.00000   60   44  480.00000  340.00000  29.2%   182 1836s
 334200 25496  340.00000   39   83  480.00000  340.00000  29.2%   182 1841s
 335622 25588     cutoff   68       480.00000  340.00000  29.2%   182 1846s
 336962 25707  370.06425   44   40  480.00000  340.00000  29.2%   182 1850s
 338429 25779  354.10484   58   57  480.00000  340.00000  29.2%   182 1855s
 341072 25873  340.00000   51   57  480.00000  340.00000  29.2%   181 1864s
 342397 25921  348.01840   42   77  480.00000  340.00000  29.2%   181 1868s
 343537 26081  380.81288   48   42  480.00000  340.00000  29.2%   181 1873s
 344100 26317  350.08857   60   84  480.00000  340.00000  29.2%   181 1877s
 345286 26503  340.00000   56   52  480.00000  340.00000  29.2%   181 1882s
 346312 26731  365.85045   65   56  480.00000  340.00000  29.2%   181 1887s
 347730 2688

 459526 32346  340.00000   62   88  480.00000  340.00000  29.2%   185 2536s
 460505 32363  350.83309   63   62  480.00000  340.00000  29.2%   185 2542s
 461523 32357     cutoff   65       480.00000  340.00000  29.2%   185 2548s
 462887 32391  340.00000   61   58  480.00000  340.00000  29.2%   185 2561s
 463899 32421  340.00000   61   47  480.00000  340.00000  29.2%   185 2567s
 465338 32454  417.36127   61   50  480.00000  340.00000  29.2%   185 2572s
 466578 32525  340.00000   49   50  480.00000  340.00000  29.2%   185 2579s
 467529 32536     cutoff   70       480.00000  340.00000  29.2%   185 2585s
 468777 32516     cutoff   63       480.00000  340.00000  29.2%   185 2592s
 470042 32533     cutoff   66       480.00000  340.00000  29.2%   185 2598s
 470992 32536  340.00000   45   54  480.00000  340.00000  29.2%   185 2604s
 472517 32557  340.00000   56   43  480.00000  340.00000  29.2%   185 2610s
 473855 32581  340.00000   48   78  480.00000  340.00000  29.2%   185 2616s
 474887 3258

...

  1115   783  340.00000   69  121          -  340.00000      -  2070  647s
  1116   783  340.00000   45  183          -  340.00000      -  2069  650s
  1117   784  380.00000   91   89          -  340.00000      -  2067  665s
  1118   785  340.00000   34  250          -  340.00000      -  2065  670s
  1119   785  340.00000   66   85          -  340.00000      -  2063  686s
  1120   786  340.00000    4   78          -  340.00000      -  2061  691s
  1121   787  340.00000   61   78          -  340.00000      -  2059  721s
  1122   790  340.00000   14  139          -  340.00000      -   387  742s
  1124   794  340.00000   15  255          -  340.00000      -   398  765s
  1128   796  340.00000   16  216          -  340.00000      -   407  779s
  1132   799  340.00000   16  343          -  340.00000      -   423  808s
  1136   802  340.00000   17  201          -  340.00000      -   446  818s
  1140   804  340.00000   17  186          -  340.00000      -   447  828s
  1144   808  340.00000  

  1459  1009  340.00000   44   48          -  340.00000      -  1935  307s
  1461  1010  340.00000   68   30          -  340.00000      -  1932  314s
  1462  1011  340.00000   28   67          -  340.00000      -  1931  315s
  1463  1012  340.00000   89   42          -  340.00000      -  1929  325s
  1465  1013  340.00000   54  117          -  340.00000      -  1927  333s
  1466  1014  620.00000   92  175          -  340.00000      -  1925  335s
  1467  1014  340.00000   82   69          -  340.00000      -  1924  342s
  1469  1016  440.00000  130   96          -  340.00000      -  1922  353s
  1470  1016  340.00000   34  204          -  340.00000      -  1920  357s
  1471  1017  340.00000   56   85          -  340.00000      -  1919  369s
  1472  1018  520.00000   89  329          -  340.00000      -  1918  374s
  1473  1018  560.00000   99   90          -  340.00000      -  1916  382s
  1474  1019  340.00000  123  219          -  340.00000      -  1915  386s
  1475  1020  340.00000  

    15    20  340.00000    4  242          -  340.00000      - 17804  270s
    30    49  340.00000    7  153          -  340.00000      - 10167  293s
    48   125  340.00000   11   28          -  340.00000      -  9164  357s
   129   344  340.00000   17  191          -  340.00000      -  5071  456s
   373   597  340.00000   60   51          -  340.00000      -  3168  560s
   670   830  340.00000   13  260          -  340.00000      -  2539  657s
  1029   831  340.00000   54   30          -  340.00000      -  2186  786s
  1031   832  340.00000    7   58          -  340.00000      -  2181  807s
  1032   833  340.00000   49   60          -  340.00000      -  2179  854s
  1033   834  340.00000   42  382          -  340.00000      -  2177  868s
  1034   834  340.00000   27   57          -  340.00000      -  2175  910s
  1035   835  340.00000   75  102          -  340.00000      -  2173  916s
  1036   836  340.00000   73   47          -  340.00000      -  2171  958s
  1037   836  340.00000  


Cutting planes:
  Cover: 42
  Implied bound: 4
  Clique: 46
  MIR: 2
  Flow cover: 32
  Zero half: 17
  RLT: 2
  Relax-and-lift: 2

Explored 1408 nodes (5255466 simplex iterations) in 2000.05 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 860 

Time limit reached
Best objective 8.600000000000e+02, best bound 3.400000000000e+02, gap 60.4651%
A solution was found after 2000.0556118488312s with 2 pollsters, 2 vehicles, and 2 days.
Solution stored in local folder.
Freeing default Gurobi environment



*** Solving instance of size: 40  ***

Using license file /Users/Andy/gurobi.lic
Academic license - for non-commercial use only
Constraints took 3.0565412044525146 seconds.
Model ready
Changed value of parameter TimeLimit to 1000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 24007 rows, 99202 columns and 401414 nonzeros
Model fingerprint: 0x994da226
Model has 8080 general constraints
Va

   73931    4.0002942e+02   0.000000e+00   1.233217e+07    246s
   74571    3.9952902e+02   0.000000e+00   6.225962e+07    251s
   75101    3.9921701e+02   0.000000e+00   8.285539e+06    256s
   75771    3.9881842e+02   0.000000e+00   2.913535e+07    261s
   76381    3.9844067e+02   0.000000e+00   6.997372e+06    266s
   76981    3.9806000e+02   0.000000e+00   9.615453e+07    271s
   77611    3.9763899e+02   0.000000e+00   1.902368e+07    276s
   78231    3.9724887e+02   0.000000e+00   1.061203e+07    281s
   78871    3.9687666e+02   0.000000e+00   1.751548e+07    286s
   79381    3.9655377e+02   0.000000e+00   5.896560e+06    290s
   80001    3.9612604e+02   0.000000e+00   1.993622e+07    295s
   80611    3.9555837e+02   0.000000e+00   1.817085e+07    300s
   81331    3.9492636e+02   0.000000e+00   8.315105e+06    306s
   81931    3.9429909e+02   0.000000e+00   7.125897e+07    311s
   82531    3.9356481e+02   0.000000e+00   1.867544e+07    315s
   83141    3.9295379e+02   0.000000e+00

In [ ]:
import matplotlib.pyplot as plt
plt.spy(mo.getA()); plt.show()